In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [193]:
sumo = {}
sumo_files = os.listdir('data_experiment1_sumo')
for file in sumo_files:
    sumo_df = pd.read_csv(f'data_experiment1_sumo/{file}')
    sumo[file.split(".")[0]] = sumo_df
print(sumo.keys())

dict_keys(['veh1_aggressive', 'veh0_normal'])


In [194]:
carla = {}
carla_files = os.listdir('data_experiment1_carla')
for file in carla_files:
    carla_df = pd.read_csv(f'data_experiment1_carla/{file}')
    carla_df.rename(columns={'latitude': 'y_pos', 'longitude': 'x_pos'}, inplace=True)
    carla_df['acc_z'] -= 9.8
    carla[file.split('.')[0]] = carla_df[carla_df['timestamp'] > 0]
print(carla.keys())

dict_keys(['veh1_aggressive', 'veh0_normal'])


In [195]:
carla['veh0_normal'].head()

,timestamp,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,compass,y_pos,x_pos
1,0.1,0.010966,0.000016,0.010205,-1.475829e-06,2.366908e-07,2.282495e-09,79.023772,42.062875,2.010618
2,0.2,0.010966,0.000016,0.010212,-3.301918e-07,-4.842556e-08,3.207580e-10,79.023772,42.062875,2.010618
3,0.3,0.010966,0.000016,0.010193,-5.678841e-07,-2.715671e-09,6.436345e-10,79.023772,42.062875,2.010618
4,0.4,4.380508,0.432030,-0.233211,1.964860e-02,-7.281843e-02,6.133208e-02,79.195162,42.062875,2.010618
5,0.5,8.410976,0.330508,-0.036365,-2.016906e-03,3.475843e-04,7.922983e-02,79.561281,42.062875,2.010620


In [196]:
sumo['veh0_normal'].head()

,timestamp,x_pos,y_pos,speed,speed_x,speed_y,acc,acc_x,acc_y,angle,acc_diff,gyro_z
0,0.1,1883.910905,6865.150060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.082803,0.000000,0.000000e+00
1,0.2,1883.911254,6865.150140,0.007165,0.001602,0.006984,0.143307,0.032035,0.139680,77.082803,0.143307,0.000000e+00
2,0.3,1883.912080,6865.150329,0.016944,0.003788,0.016515,0.195566,0.043717,0.190617,77.082803,0.195566,0.000000e+00
3,0.4,1883.913551,6865.150666,0.030181,0.006747,0.029417,0.264741,0.059181,0.258041,77.082803,0.069175,0.000000e+00
4,0.5,1883.915886,6865.151202,0.047912,0.010710,0.046699,0.354618,0.079272,0.345644,77.082803,0.089878,-1.043610e-14


The time series may have different lengths because of the routing and simulation of each simulator.

In [197]:
for (sumo_k, sumo_v), (carla_k, carla_v) in zip(sumo.items(), carla.items()):
    print(f'Sumo {sumo_k} size: {len(sumo_v)}. Carla {sumo_k} size: {len(carla_v)}')


Sumo veh1_aggressive size: 3484. Carla veh1_aggressive size: 8277
Sumo veh0_normal size: 9727. Carla veh0_normal size: 8908


Note que gyro_z não é obtido diretamente pelo SUMO, aqui está sendo realizada uma aproximação a partir da variação nos ângulos

In [198]:
def plot_df(sumo_df=None, carla_df=None, idx="", uah_df_imu=None, uah_df_gnss=None, save_path=None):
    fig, axes = plt.subplots(4, 2, figsize=(20, 18))
    axes = axes.flatten()
    alpha = 0.6

    # Plot acceleration X
    if sumo_df is not None:
        axes[0].plot(sumo_df['timestamp'], sumo_df['acc_x'], label='Sumo Acc X', alpha=alpha)
    if carla_df is not None:
        axes[0].plot(carla_df['timestamp'], carla_df['acc_x'], label='Carla Acc X', alpha=alpha)
    if uah_df_imu is not None:
        axes[0].plot(uah_df_imu['timestamp'], uah_df_imu['acc_x'], label='UAH Acc X', color='red', alpha=0.7)
    axes[0].set_title("Acceleration X over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    # Plot acceleration Y
    if sumo_df is not None:
        axes[1].plot(sumo_df['timestamp'], sumo_df['acc_y'], label='Sumo Acc Y', alpha=alpha)
    if carla_df is not None:
        axes[1].plot(carla_df['timestamp'], carla_df['acc_y'], label='Carla Acc Y', alpha=alpha)
    if uah_df_imu is not None:
        axes[1].plot(uah_df_imu['timestamp'], uah_df_imu['acc_y'], label='UAH Acc Y', color='red', alpha=0.7)
    axes[1].set_title("Acceleration Y over Time")
    axes[1].set_ylabel("Acceleration (m/s²)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot gyroscope Z
    if sumo_df is not None:
        axes[2].plot(sumo_df['timestamp'], sumo_df['gyro_z'], label='Sumo Gyro Z', alpha=alpha)
    if carla_df is not None:
        axes[2].plot(carla_df['timestamp'], carla_df['gyro_z'], label='Carla Gyro Z', alpha=alpha)
    if uah_df_imu is not None and 'Yaw' in uah_df_imu.columns:
        axes[2].plot(uah_df_imu['timestamp'], uah_df_imu['Yaw'], label='UAH Yaw', color='red', alpha=0.7)
    axes[2].set_title("Gyroscope Z over Time")
    axes[2].set_ylabel("Angular Velocity (rad/s)")
    axes[2].legend()
    axes[2].grid(True)

    axes_count = 3
    # Plot angle
    if uah_df_gnss is None:
        if sumo_df is not None:
            axes[axes_count].plot(sumo_df['timestamp'], sumo_df['angle'], label='Sumo Angle', alpha=alpha)
        if carla_df is not None:
            axes[axes_count].plot(carla_df['timestamp'], carla_df['compass'], label='Carla Angle', alpha=alpha)
        if uah_df_imu is not None and 'Yaw' in uah_df_imu.columns:
            axes[axes_count].plot(uah_df_imu['timestamp'], uah_df_imu['Yaw'], label='UAH Yaw', color='red', alpha=0.7)
        axes[axes_count].set_title("Angle over Time")
        axes[axes_count].set_ylabel("Angle (rad)")
        axes[axes_count].legend()
        axes[axes_count].grid(True)
        axes_count += 1

    # Plot Sumo position
    if sumo_df is not None:
        scatter = axes[axes_count].scatter(sumo_df['x_pos'], sumo_df['y_pos'], c=sumo_df['timestamp'], cmap='viridis', s=10, label='Sumo')
        axes[axes_count].set_title("Sumo Position over Time")
        axes[axes_count].set_xlabel("X position")
        axes[axes_count].set_ylabel("Y position")
        axes[axes_count].grid(True)
        cbar = fig.colorbar(scatter, ax=axes[axes_count])
        cbar.set_label("Time (s)")
        axes_count += 1

    # Plot Carla position
    if carla_df is not None:
        scatter = axes[axes_count].scatter(carla_df['x_pos'], carla_df['y_pos'], c=carla_df['timestamp'], cmap='viridis', s=10, label='Carla')
        axes[axes_count].set_title("Carla Position over Time")
        axes[axes_count].set_xlabel("X position")
        axes[axes_count].set_ylabel("Y position")
        axes[axes_count].grid(True)
        cbar = fig.colorbar(scatter, ax=axes[axes_count])
        cbar.set_label("Time (s)")
        axes_count += 1

    # Plot UAH position if available
    if uah_df_gnss is not None and 'lat' in uah_df_gnss.columns and 'lon' in uah_df_gnss.columns and 'timestamp' in uah_df_gnss.columns:
        scatter = axes[axes_count].scatter(uah_df_gnss['lon'], uah_df_gnss['lat'], c=uah_df_gnss['timestamp'], cmap='viridis', s=10, label='UAH', alpha=0.7)
        axes[axes_count].set_title("UAH Position over Time")
        axes[axes_count].set_xlabel("Longitude")
        axes[axes_count].set_ylabel("Latitude")
        axes[axes_count].grid(True)
        cbar = fig.colorbar(scatter, ax=axes[axes_count])
        cbar.set_label("Time (s)")
        axes_count += 1

    # Remove unused axes if any
    if axes_count < 8:
        for i in range(axes_count, 8):
            fig.delaxes(axes[i])

    fig.tight_layout(rect=[0, 0.03, 1, 0.96])

    if uah_df_imu is not None:
        if carla_df is not None:
            if sumo_df is not None:
                fig.suptitle(f'Comparison of SUMO, CARLA, and UAH Data for {idx}', fontsize=18)
            else:
                fig.suptitle(f'Comparison of CARLA and UAH Data for {idx}', fontsize=18)
        else:
            if sumo_df is not None:
                fig.suptitle(f'Comparison of SUMO and UAH Data for {idx}', fontsize=18)
            else:
                fig.suptitle(f'UAH Data for {idx}', fontsize=18)
    else:
        if carla_df is not None:
            if sumo_df is not None:
                fig.suptitle(f'Comparison of SUMO and CARLA Data for {idx}', fontsize=18)
            else:
                fig.suptitle(f'CARLA Data for {idx}', fontsize=18)
        else:
            if sumo_df is not None:
                fig.suptitle(f'SUMO Data for {idx}', fontsize=18)
            else:
                fig.suptitle(f'No data available for {idx}', fontsize=18)
    
    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [9]:
os.makedirs('plots', exist_ok=True)
for veh in sumo.keys():
    plot_df(sumo[veh], carla[veh], veh, save_path=f'plots/{veh}.png')

In [100]:
def plot_histograms(sumo_df, carla_df, idx, uah_df_imu = None, bins=50, save_path=None):
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(20, 15))
    axes = axes.flatten()

    # Plot acceleration X
    axes[0].hist(sumo_df['acc_x'], bins=bins, alpha=0.5, label='Sumo Acc X', density=True, color='blue')
    axes[0].hist(carla_df['acc_x'], bins=bins, alpha=0.5, label='Carla Acc X', density=True, color='orange')
    sumo_df['acc_x'].plot(kind='kde', ax=axes[0], label='Sumo Acc X KDE', color='blue')
    carla_df['acc_x'].plot(kind='kde', ax=axes[0], label='Carla Acc X KDE', color='orange')
    if uah_df_imu is not None and 'acc_x' in uah_df_imu.columns:
        axes[0].hist(uah_df_imu['acc_x'], bins=bins, alpha=0.5, label='UAH Acc X', density=True, color='green')
        uah_df_imu['acc_x'].plot(kind='kde', ax=axes[0], label='UAH Acc X KDE', color='green')
    axes[0].set_title("Acceleration X Histogram")
    axes[0].set_ylabel("Density")
    axes[0].legend()
    axes[0].grid(True)

    # Plot acceleration Y
    axes[1].hist(sumo_df['acc_y'], bins=bins, alpha=0.5, label='Sumo Acc Y', density=True, color='blue')
    axes[1].hist(carla_df['acc_y'], bins=bins, alpha=0.5, label='Carla Acc Y', density=True, color='orange')
    sumo_df['acc_y'].plot(kind='kde', ax=axes[1], label='Sumo Acc Y KDE', color='blue')
    carla_df['acc_y'].plot(kind='kde', ax=axes[1], label='Carla Acc Y KDE', color='orange')
    if uah_df_imu is not None and 'acc_y' in uah_df_imu.columns:
        axes[1].hist(uah_df_imu['acc_y'], bins=bins, alpha=0.5, label='UAH Acc Y', density=True, color='green')
        uah_df_imu['acc_y'].plot(kind='kde', ax=axes[1], label='UAH Acc Y KDE', color='green')
    axes[1].set_title("Acceleration Y Histogram")
    axes[1].set_ylabel("Density")
    axes[1].legend()
    axes[1].grid(True)

    # Plot gyroscope Z
    axes[2].hist(sumo_df['gyro_z'], bins=bins, alpha=0.5, label='Sumo Gyro Z', density=True, color='blue')
    axes[2].hist(carla_df['gyro_z'], bins=bins, alpha=0.5, label='Carla Gyro Z', density=True, color='orange')
    sumo_df['gyro_z'].plot(kind='kde', ax=axes[2], label='Sumo Gyro Z KDE', color='blue')
    carla_df['gyro_z'].plot(kind='kde', ax=axes[2], label='Carla Gyro Z KDE', color='orange')
    if uah_df_imu is not None and 'Yaw' in uah_df_imu.columns:
        axes[2].hist(uah_df_imu['Yaw'], bins=bins, alpha=0.5, label='UAH Yaw', density=True, color='green')
        uah_df_imu['Yaw'].plot(kind='kde', ax=axes[2], label='UAH Yaw KDE', color='green')
    axes[2].set_title("Gyroscope Z Histogram")
    axes[2].set_ylabel("Density")
    axes[2].legend()
    axes[2].grid(True)

    # Plot angle
    axes[3].hist(sumo_df['angle'], bins=bins, alpha=0.5, label='Sumo Angle', density=True, color='blue')
    axes[3].hist(carla_df['compass'], bins=bins, alpha=0.5, label='Carla Angle', density=True, color='orange')
    sumo_df['angle'].plot(kind='kde', ax=axes[3], label='Sumo Angle KDE', color='blue')
    carla_df['compass'].plot(kind='kde', ax=axes[3], label='Carla Angle KDE', color='orange')
    axes[3].set_title("Angle Histogram")
    axes[3].set_ylabel("Density")
    axes[3].legend()
    axes[3].grid(True)

    # Set the main title
    fig.tight_layout(rect=[0, 0.03, 1, 0.96])  # Leave space at the top for suptitle
    fig.suptitle(f'Histogram Comparison of SUMO, CARLA, and UAH Data for {idx}', fontsize=18)
    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [11]:
os.makedirs('plots_hist', exist_ok=True)
for veh in sumo.keys():
    plot_histograms(sumo[veh], carla[veh], veh, save_path=f'plots_hist/hist_{veh}.png')

## UAH-Driveset

In [199]:
def getData(driver, specifier, sensor):

    if sensor == 'acc':
        for i in os.listdir(f'UAH-DRIVESET-v1/{driver}'):
            if i.endswith(specifier):
                data = pd.read_csv(f'UAH-DRIVESET-v1/{driver}/{i}/RAW_ACCELEROMETERS.txt', sep=' ', header=None, names=[
                    'timestamp',
                    'system_active',
                    'acc_x',
                    'acc_y',
                    'acc_z',
                    'acc_x_KF',
                    'acc_y_KF',
                    'acc_z_KF',
                    'Roll',
                    'Pitch',
                    'Yaw'
                ], usecols=range(11))
                data = data.drop(['system_active'], axis=1)
                data['acc'] = np.sqrt(data['acc_x']**2 + data['acc_y']**2 + data['acc_z']**2)
                return data
            
    elif sensor == 'gps':
        for i in os.listdir(f'UAH-DRIVESET-v1/{driver}'):
            if i.endswith(specifier):
                data = pd.read_csv(f'UAH-DRIVESET-v1/{driver}/{i}/RAW_GPS.txt', sep=' ', header=None, names=[
                    'timestamp',
                    'speed',
                    'lat',
                    'lon',
                    'altitude',
                    'vert_accuracy',
                    'horiz_accuracy',
                    'course',
                    'difcourse',
                ], usecols=range(9))
                return data

In [200]:
def read_gps(drivers):
    normal = pd.DataFrame()
    aggressive = pd.DataFrame()
    drowsy = pd.DataFrame()
    for driver in drivers:
        normal = pd.concat([normal, getData(driver, 'NORMAL1-SECONDARY', sensor='gps')], axis=0)
        # normal = pd.concat([normal, getData(driver, 'NORMAL2-SECONDARY', sensor='gps')], axis=0)
        aggressive = pd.concat([aggressive, getData(driver, 'AGGRESSIVE-SECONDARY', sensor='gps')], axis=0)
        drowsy = pd.concat([drowsy, getData(driver, 'DROWSY-SECONDARY', sensor='gps')], axis=0)

    df_gps = {}
    df_gps['normal'] = normal
    df_gps['aggressive'] = aggressive
    df_gps['drowsy'] = drowsy

    return df_gps

In [201]:
def read_accelerometer(drivers):
    normal = pd.DataFrame()
    aggressive = pd.DataFrame()
    drowsy = pd.DataFrame()
    for driver in drivers:
        normal = pd.concat([normal, getData(driver, 'NORMAL1-SECONDARY', sensor='acc')], axis=0)
        # normal = pd.concat([normal, getData(driver, 'NORMAL2-SECONDARY', sensor='acc')], axis=0)
        aggressive = pd.concat([aggressive, getData(driver, 'AGGRESSIVE-SECONDARY', sensor='acc')], axis=0)
        drowsy = pd.concat([drowsy, getData(driver, 'DROWSY-SECONDARY', sensor='acc')], axis=0)

    df_accelerometer = {}
    df_accelerometer['normal'] = normal
    df_accelerometer['aggressive'] = aggressive
    df_accelerometer['drowsy'] = drowsy

    return df_accelerometer

Reading from only one driver and one run to be able to compare

In [202]:
drivers = ['D1']

In [203]:
df_acc = read_accelerometer(drivers)
df_gps = read_gps(drivers)

In [204]:
df_gps['normal'].head()

,timestamp,speed,lat,lon,altitude,vert_accuracy,horiz_accuracy,course,difcourse
0,7.85,65.2,40.512787,-3.404477,612.7,4,5,331.9,0.000
1,8.83,64.5,40.512924,-3.404577,612.5,4,5,331.9,0.000
2,9.82,63.6,40.513065,-3.404680,612.9,4,5,330.8,1.055
3,10.80,62.2,40.513210,-3.404772,613.3,4,5,330.8,1.055
4,11.80,60.9,40.513348,-3.404868,613.5,3,5,330.1,0.703


In [205]:
df_acc['normal'].head()

,timestamp,acc_x,acc_y,acc_z,acc_x_KF,acc_y_KF,acc_z_KF,Roll,Pitch,Yaw,acc
0,6.94,0.017,-0.011,0.018,-0.005,0.008,0.018,-1.523,0.015,0.012,0.027092
1,7.03,0.046,0.007,0.019,0.016,-0.002,0.018,-1.522,0.012,0.012,0.050259
2,7.14,0.052,-0.016,0.027,0.037,-0.005,0.018,-1.520,0.014,0.011,0.060737
3,7.24,0.015,-0.016,0.026,0.038,-0.009,0.024,-1.523,0.014,0.011,0.034015
4,7.34,-0.014,-0.017,0.040,0.012,-0.016,0.032,-1.525,0.012,0.011,0.045662


Simulated only up to lat, lon = 40.554174, -3.497402 because of simulation problems. Let's trim the dataset in this position

In [206]:
def trim_gnss(df_gnss, lat, lon, lat_margin=0.0001, lon_margin=0.0001):

    # Define bounds
    lat_min = lat - lat_margin
    lat_max = lat + lat_margin
    lon_min = lon - lon_margin
    lon_max = lon + lon_margin

    # Find the first index within the interval
    condition = (
        (df_gnss['lat'] >= lat_min) & (df_gnss['lat'] <= lat_max) &
        (df_gnss['lon'] >= lon_min) & (df_gnss['lon'] <= lon_max)
    )

    # Get the first matching index
    matching_indices = df_gnss.index[condition]

    if not matching_indices.empty:
        closest_idx = matching_indices[0]
        df_trimmed = df_gnss.loc[closest_idx:].copy()

    else:
        print("No point found within the specified interval.")
        df_trimmed = df_gnss.copy()  # or handle differently
    
    return df_trimmed

In [207]:
max_lat, max_lon = 40.554174, -3.497402
# Find all rows where lat and lon are within a small range of max_lat and max_lon
df_gps['normal'] = trim_gnss(df_gps['normal'], max_lat, max_lon, lat_margin=0.0005, lon_margin=0.0005)
df_gps['aggressive'] = trim_gnss(df_gps['aggressive'], max_lat, max_lon, lat_margin=0.0001, lon_margin=0.0001)

In [208]:
def plot_gps(df_gps):
    fig, ax = plt.subplots(figsize=(15, 8))
    scatter = ax.scatter(df_gps['lon'], df_gps['lat'], c=df_gps['timestamp'], cmap='viridis', s=10)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label("Time (s)")
    ax.set_title("GPS Position over Time")
    plt.grid(True)
    plt.show()

In [209]:
def plot_sumo_uah(sumo_df, uah_df_gnss, uah_df_imu, idx, save_path=None):
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    axes = axes.flatten()
    alpha = 0.6

    # Plot acceleration X
    axes[0].plot(sumo_df['timestamp'], sumo_df['acc_x'], label='Sumo Acc X', alpha=alpha)
    axes[0].plot(uah_df_imu['timestamp'], uah_df_imu['acc_x'], label='UAH Acc X', color='red', alpha=0.7)
    axes[0].set_title("Acceleration X over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    # Plot acceleration Y
    axes[1].plot(sumo_df['timestamp'], sumo_df['acc_y'], label='Sumo Acc Y', alpha=alpha)
    axes[1].plot(uah_df_imu['timestamp'], uah_df_imu['acc_y'], label='UAH Acc Y', color='red', alpha=0.7)
    axes[1].set_title("Acceleration Y over Time")
    axes[1].set_ylabel("Acceleration (m/s²)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot gyroscope Z
    axes[2].plot(sumo_df['timestamp'], sumo_df['gyro_z'], label='Sumo Gyro Z', alpha=alpha)
    axes[2].plot(uah_df_imu['timestamp'], uah_df_imu['Yaw'], label='UAH Yaw', color='red', alpha=0.7)
    axes[2].set_title("Gyroscope Z over Time")
    axes[2].set_ylabel("Angular Velocity (rad/s)")
    axes[2].legend()
    axes[2].grid(True)

    fig.tight_layout(rect=[0, 0.03, 1, 0.94])

    fig.suptitle(f'Comparison of SUMO and UAH Data data for {veh.split('_')[-1]} vehicle', fontsize=16)

    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes = axes.flatten()

    # Plot Sumo position
    scatter = axes[0].scatter(sumo_df['x_pos'], sumo_df['y_pos'], c=sumo_df['timestamp'], cmap='viridis', s=10, label='Sumo')
    axes[0].set_title("Sumo Position over Time")
    axes[0].set_xlabel("X position")
    axes[0].set_ylabel("Y position")
    axes[0].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[0])
    cbar.set_label("Time (s)")

    # Plot UAH position if available
    scatter = axes[1].scatter(uah_df_gnss['lon'], uah_df_gnss['lat'], c=uah_df_gnss['timestamp'], cmap='viridis', s=10, label='UAH')
    axes[1].set_title("UAH Position over Time")
    axes[1].set_xlabel("Longitude")
    axes[1].set_ylabel("Latitude")
    axes[1].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[1])
    cbar.set_label("Time (s)")

    if save_path:
        plt.savefig(f'{save_path}_position.png')
        plt.close(fig)
    else:
        plt.show()

In [210]:
def plot_carla_uah(carla_df, uah_df_imu, uah_df_gnss, veh, save_path=None):
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    axes = axes.flatten()
    alpha = 0.6

    # Plot acceleration X
    axes[0].plot(carla_df['timestamp'], carla_df['acc_x'], label='Carla Acc X', alpha=alpha)
    axes[0].plot(uah_df_imu['timestamp'], uah_df_imu['acc_x'], label='UAH Acc X', color='red', alpha=0.7)
    axes[0].set_title("Acceleration X over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    # Plot acceleration Y
    axes[1].plot(carla_df['timestamp'], carla_df['acc_y'], label='Carla Acc Y', alpha=alpha)
    axes[1].plot(uah_df_imu['timestamp'], uah_df_imu['acc_y'], label='UAH Acc Y', color='red', alpha=0.7)
    axes[1].set_title("Acceleration Y over Time")
    axes[1].set_ylabel("Acceleration (m/s²)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot acceleration Z
    axes[2].plot(carla_df['timestamp'], carla_df['acc_z'], label='Carla Acc Z', alpha=alpha)
    axes[2].plot(uah_df_imu['timestamp'], uah_df_imu['acc_z'], label='UAH Acc Z', color='red', alpha=0.7)
    axes[2].set_title("Acceleration Z over Time")
    axes[2].set_ylabel("Acceleration (m/s²)")
    axes[2].legend()
    axes[2].grid(True)

    # Plot gyroscope X
    axes[3].plot(carla_df['timestamp'], carla_df['gyro_x'], label='Carla Gyro X', alpha=alpha)
    axes[3].plot(uah_df_imu['timestamp'], uah_df_imu['Roll'], label='UAH Roll', color='red', alpha=0.7)
    axes[3].set_title("Gyroscope X over Time")
    axes[3].set_ylabel("Angular Velocity (rad/s)")
    axes[3].legend()
    axes[3].grid(True)

    # Plot gyroscope Y
    axes[4].plot(carla_df['timestamp'], carla_df['gyro_y'], label='Carla Gyro Y', alpha=alpha)
    axes[4].plot(uah_df_imu['timestamp'], uah_df_imu['Pitch'], label='UAH Pitch', color='red', alpha=0.7)
    axes[4].set_title("Gyroscope Y over Time")
    axes[4].set_ylabel("Angular Velocity (rad/s)")
    axes[4].legend()
    axes[4].grid(True)

    # Plot gyroscope Z
    axes[5].plot(carla_df['timestamp'], carla_df['gyro_z'], label='Carla Gyro Z', alpha=alpha)
    axes[5].plot(uah_df_imu['timestamp'], uah_df_imu['Yaw'], label='UAH Yaw', color='red', alpha=0.7)
    axes[5].set_title("Gyroscope Z over Time")
    axes[5].set_ylabel("Angular Velocity (rad/s)")
    axes[5].legend()
    axes[5].grid(True)

    fig.tight_layout(rect=[0, 0.03, 1, 0.94])

    fig.suptitle(f'Comparison of CARLA and UAH Data for {veh.split('_')[-1]} vehicle', fontsize=16)

    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes = axes.flatten()

    # Plot Carla position
    scatter = axes[0].scatter(carla_df['x_pos'], carla_df['y_pos'], c=carla_df['timestamp'], cmap='viridis', s=10, label='Carla')
    axes[0].set_title("Carla Position over Time")
    axes[0].set_xlabel("X position")
    axes[0].set_ylabel("Y position")
    axes[0].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[0])
    cbar.set_label("Time (s)")

    # Plot UAH position if available
    scatter = axes[1].scatter(uah_df_gnss['lon'], uah_df_gnss['lat'], c=uah_df_gnss['timestamp'], cmap='viridis', s=10, label='UAH')
    axes[1].set_title("UAH Position over Time")
    axes[1].set_xlabel("Longitude")
    axes[1].set_ylabel("Latitude")
    axes[1].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[1])
    cbar.set_label("Time (s)")

    if save_path:
        plt.savefig(f'{save_path}_position.png')
        plt.close(fig)
    else:
        plt.show()

In [211]:
def hist_sumo_uah(sumo_df, uah_df_imu, uah_df_gnss, veh, save_path=None):
    # Create subplots
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    axes = axes.flatten()

    # Plot acceleration X
    axes[0].hist(sumo_df['acc_x'], bins=50, alpha=0.5, label='Sumo Acc X', density=True, color='blue')
    axes[0].hist(uah_df_imu['acc_x'], bins=50, alpha=0.5, label='UAH Acc X', density=True, color='green')
    sumo_df['acc_x'].plot(kind='kde', ax=axes[0], label='Sumo Acc X KDE', color='blue')
    uah_df_imu['acc_x'].plot(kind='kde', ax=axes[0], label='UAH Acc X KDE', color='green')
    axes[0].set_title("Acceleration X Histogram")
    axes[0].set_ylabel("Density")
    axes[0].legend()
    axes[0].grid(True)

    # Plot acceleration Y
    axes[1].hist(sumo_df['acc_y'], bins=50, alpha=0.5, label='Sumo Acc Y', density=True, color='blue')
    axes[1].hist(uah_df_imu['acc_y'], bins=50, alpha=0.5, label='UAH Acc Y', density=True, color='green')
    sumo_df['acc_y'].plot(kind='kde', ax=axes[1], label='Sumo Acc Y KDE', color='blue')
    uah_df_imu['acc_y'].plot(kind='kde', ax=axes[1], label='UAH Acc Y KDE', color='green')
    axes[1].set_title("Acceleration Y Histogram")
    axes[1].set_ylabel("Density")
    axes[1].legend()
    axes[1].grid(True)

    # Plot gyroscope Z
    axes[2].hist(sumo_df['gyro_z'], bins=50, alpha=0.5, label='Sumo Gyro Z', density=True, color='blue')
    axes[2].hist(uah_df_imu['Yaw'], bins=50, alpha=0.5, label='UAH Yaw', density=True, color='green')
    sumo_df['gyro_z'].plot(kind='kde', ax=axes[2], label='Sumo Gyro Z KDE', color='blue')
    uah_df_imu['Yaw'].plot(kind='kde', ax=axes[2], label='UAH Yaw KDE', color='green')
    axes[2].set_title("Gyroscope Z Histogram")
    axes[2].set_ylabel("Density")
    axes[2].legend()
    axes[2].grid(True)

    fig.tight_layout(rect=[0, 0.03, 1, 0.94])  # Leave space at the top for suptitle
    fig.suptitle(f'Histogram Comparison of SUMO and UAH Data for {veh.split("_")[-1]} vehicle', fontsize=16)
    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()

In [212]:
def hist_carla_uah(carla_df, uah_df_imu, uah_df_gnss, veh, save_path=None):
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    axes = axes.flatten()
    alpha = 0.6

    # Acceleration X histogram
    axes[0].hist(carla_df['acc_x'], bins=50, alpha=0.5, label='Carla Acc X', density=True, color='blue')
    axes[0].hist(uah_df_imu['acc_x'], bins=50, alpha=0.5, label='UAH Acc X', density=True, color='green')
    carla_df['acc_x'].plot(kind='kde', ax=axes[0], label='Carla Acc X KDE', color='blue')
    uah_df_imu['acc_x'].plot(kind='kde', ax=axes[0], label='UAH Acc X KDE', color='green')
    axes[0].set_title("Acceleration X Histogram")
    axes[0].set_ylabel("Density")
    axes[0].legend()
    axes[0].grid(True)

    # Acceleration Y histogram
    axes[1].hist(carla_df['acc_y'], bins=50, alpha=0.5, label='Carla Acc Y', density=True, color='blue')
    axes[1].hist(uah_df_imu['acc_y'], bins=50, alpha=0.5, label='UAH Acc Y', density=True, color='green')
    carla_df['acc_y'].plot(kind='kde', ax=axes[1], label='Carla Acc Y KDE', color='blue')
    uah_df_imu['acc_y'].plot(kind='kde', ax=axes[1], label='UAH Acc Y KDE', color='green')
    axes[1].set_title("Acceleration Y Histogram")
    axes[1].set_ylabel("Density")
    axes[1].legend()
    axes[1].grid(True)

    # Acceleration Z histogram
    axes[2].hist(carla_df['acc_z'], bins=50, alpha=0.5, label='Carla Acc Z', density=True, color='blue')
    axes[2].hist(uah_df_imu['acc_z'], bins=50, alpha=0.5, label='UAH Acc Z', density=True, color='green')
    carla_df['acc_z'].plot(kind='kde', ax=axes[2], label='Carla Acc Z KDE', color='blue')
    uah_df_imu['acc_z'].plot(kind='kde', ax=axes[2], label='UAH Acc Z KDE', color='green')
    axes[2].set_title("Acceleration Z Histogram")
    axes[2].set_ylabel("Density")
    axes[2].legend()
    axes[2].grid(True)

    # Gyroscope X histogram
    axes[3].hist(carla_df['gyro_x'], bins=50, alpha=0.5, label='Carla Gyro X', density=True, color='blue')
    axes[3].hist(uah_df_imu['Roll'], bins=50, alpha=0.5, label='UAH Roll', density=True, color='green')
    carla_df['gyro_x'].plot(kind='kde', ax=axes[3], label='Carla Gyro X KDE', color='blue')
    uah_df_imu['Roll'].plot(kind='kde', ax=axes[3], label='UAH Roll KDE', color='green')
    axes[3].set_title("Gyroscope X Histogram")
    axes[3].set_ylabel("Density")
    axes[3].legend()
    axes[3].grid(True)

    # Gyroscope Y histogram
    axes[4].hist(carla_df['gyro_y'], bins=50, alpha=0.5, label='Carla Gyro Y', density=True, color='blue')
    axes[4].hist(uah_df_imu['Pitch'], bins=50, alpha=0.5, label='UAH Pitch', density=True, color='green')
    carla_df['gyro_y'].plot(kind='kde', ax=axes[4], label='Carla Gyro Y KDE', color='blue')
    uah_df_imu['Pitch'].plot(kind='kde', ax=axes[4], label='UAH Pitch KDE', color='green')
    axes[4].set_title("Gyroscope Y Histogram")
    axes[4].set_ylabel("Density")
    axes[4].legend()
    axes[4].grid(True)

    # Gyroscope Z histogram
    axes[5].hist(carla_df['gyro_z'], bins=50, alpha=0.5, label='Carla Gyro Z', density=True, color='blue')
    axes[5].hist(uah_df_imu['Yaw'], bins=50, alpha=0.5, label='UAH Yaw', density=True, color='green')
    carla_df['gyro_z'].plot(kind='kde', ax=axes[5], label='Carla Gyro Z KDE', color='blue')
    uah_df_imu['Yaw'].plot(kind='kde', ax=axes[5], label='UAH Yaw KDE', color='green')
    axes[5].set_title("Gyroscope Z Histogram")
    axes[5].set_ylabel("Density")
    axes[5].legend()
    axes[5].grid(True)

    fig.tight_layout(rect=[0, 0.03, 1, 0.94])

    fig.suptitle(f'Comparison of CARLA and UAH Data for {veh.split('_')[-1]} vehicle', fontsize=16)

    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [213]:
os.makedirs('plots_hist', exist_ok=True)
os.makedirs('plots_uah', exist_ok=True)
for id in carla.keys():
    if id.split('_')[1] == 'normal':
        plot_carla_uah(carla[id], uah_df_imu=df_acc['normal'], uah_df_gnss=df_gps['normal'], veh=id, save_path=f'plots_uah/carla_{id}.png')
        hist_carla_uah(carla[id], uah_df_imu=df_acc['normal'], uah_df_gnss=df_gps['normal'], veh=id, save_path=f'plots_hist/carla_{id}.png')
    else:
        plot_carla_uah(carla[id], uah_df_imu=df_acc['aggressive'], uah_df_gnss=df_gps['aggressive'], veh=id, save_path=f'plots_uah/carla_{id}.png')
        hist_carla_uah(carla[id], uah_df_imu=df_acc['aggressive'], uah_df_gnss=df_gps['aggressive'], veh=id, save_path=f'plots_hist/carla_{id}.png')

In [214]:
for id in sumo.keys():
    if id.split('_')[1] == 'normal':
        plot_sumo_uah(sumo[id], uah_df_gnss=df_gps['normal'], uah_df_imu=df_acc['normal'], idx=id, save_path=f'plots_uah/sumo_{id}.png')
        hist_sumo_uah(sumo[id], uah_df_imu=df_acc['normal'], uah_df_gnss=df_gps['normal'], veh=id, save_path=f'plots_hist/sumo_{id}.png')
    else:
        plot_sumo_uah(sumo[id], uah_df_gnss=df_gps['aggressive'], uah_df_imu=df_acc['aggressive'], idx=id, save_path=f'plots_uah/sumo_{id}.png')
        hist_sumo_uah(sumo[id], uah_df_imu=df_acc['aggressive'], uah_df_gnss=df_gps['aggressive'], veh=id, save_path=f'plots_hist/sumo_{id}.png')

# Comparison Metrics

In [215]:
import numpy as np
from scipy.stats import entropy, wasserstein_distance, ks_2samp
from scipy.spatial.distance import jensenshannon

In [222]:
def compare_distributions(carla_data, sumo_data, bins=100):
    # Define common bin edges
    min_val = min(carla_data.min(), sumo_data.min())
    max_val = max(carla_data.max(), sumo_data.max())
    hist_bins = np.linspace(min_val, max_val, bins + 1)

    # Histogram with density=True to approximate PDFs
    carla_hist, _ = np.histogram(carla_data, bins=hist_bins, density=True)
    sumo_hist, _ = np.histogram(sumo_data, bins=hist_bins, density=True)

    # Avoid zero entries to prevent divide-by-zero in KL
    carla_hist += 1e-10
    sumo_hist += 1e-10

    # Normalize (just in case)
    carla_hist /= carla_hist.sum()
    sumo_hist /= sumo_hist.sum()

    # KL Divergence (not symmetric)
    kl_div = entropy(carla_hist, sumo_hist)

    # Jensen-Shannon Distance (symmetric and bounded)
    js_dist = jensenshannon(carla_hist, sumo_hist)

    # Wasserstein Distance (Earth Mover’s Distance)
    wasser_dist = wasserstein_distance(carla_data, sumo_data)

    # Kolmogorov–Smirnov Test (returns D-statistic and p-value)
    ks_stat, ks_pval = ks_2samp(carla_data, sumo_data)

    # Define which metrics are "bigger is better" (↑) or "smaller is better" (↓)
    # For these metrics, smaller is better: KL, JS, Wasserstein, KS Statistic
    # For KS p-value, bigger is better
    return {
        "KL Divergence ↓": kl_div,
        "Jensen-Shannon Distance ↓": js_dist,
        "Wasserstein Distance ↓": wasser_dist,
        "KS Statistic ↓": ks_stat,
        "KS p-value ↑": ks_pval
    }

In [270]:
sensors = ['acc_x', 'acc_y', 'gyro_z']
metrics_carla_sumo = {}
for id in carla.keys():
    beh = id.split('_')[1]
    
    metrics_carla_sumo[beh] = {}
    for sensor in sensors:
        metrics_carla_sumo[beh][sensor] = compare_distributions(carla[id][sensor], sumo[id][sensor])

In [271]:
sensors = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
metrics_carla_uah = {}
for id in carla.keys():
    beh = id.split('_')[1]
    if beh == 'normal':
        uah = df_acc['normal'].copy()
    else:
        uah = df_acc['aggressive'].copy()
    uah.rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'}, inplace=True)

    metrics_carla_uah[beh] = {}
    for sensor in sensors:
        metrics_carla_uah[beh][sensor] = compare_distributions(carla[id][sensor], uah[sensor])

In [276]:
sensors = ['acc_x', 'acc_y', 'gyro_z']
metrics_sumo_uah = {}
for id in sumo.keys():
    beh = id.split('_')[1]
    if beh == 'normal':
        uah = df_acc['normal'].copy()
    else:
        uah = df_acc['aggressive'].copy()
    uah.rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'}, inplace=True)

    metrics_sumo_uah[beh] = {}
    for sensor in sensors:
        metrics_sumo_uah[beh][sensor] = compare_distributions(sumo[id][sensor], uah[sensor])

In [273]:
# Getting a second real driver to compare with
df_acc_2 = read_accelerometer(['D2'])
df_gps_2 = read_gps(['D2'])
max_lat, max_lon = 40.554174, -3.497402
# Find all rows where lat and lon are within a small range of max_lat and max_lon
df_gps_2['normal'] = trim_gnss(df_gps_2['normal'], max_lat, max_lon, lat_margin=0.0005, lon_margin=0.0005)
df_gps_2['aggressive'] = trim_gnss(df_gps_2['aggressive'], max_lat, max_lon, lat_margin=0.0001, lon_margin=0.0001)

sensors = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
metrics_uah_uah = {}
for id in ['normal', 'aggressive']:
    if id == 'normal':
        uah_1 = df_acc['normal'].copy()
        uah_2 = df_acc_2['normal'].copy()
    else:
        uah_1 = df_acc['aggressive'].copy()
        uah_2 = df_acc_2['aggressive'].copy()

    uah_1 = df_acc[id].rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'})
    uah_2 = df_acc_2[id].rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'})

    metrics_uah_uah[id] = {}
    for sensor in sensors:
        metrics_uah_uah[id][sensor] = compare_distributions(uah_1[sensor], uah_2[sensor])

In [282]:
# Convert metrics dict to DataFrame for tabular display
print("Metrics for CARLA vs SUMO (normal):")
metrics_carla_sumo_df = pd.DataFrame(metrics_carla_sumo['normal']).T
display(metrics_carla_sumo_df)

print("Metrics for CARLA vs SUMO (aggressive):")
metrics_carla_sumo_df = pd.DataFrame(metrics_carla_sumo['aggressive']).T
display(metrics_carla_sumo_df)

print("Metrics for CARLA vs UAH (normal):")
metrics_carla_uah_df = pd.DataFrame(metrics_carla_uah['normal']).T
display(metrics_carla_uah_df)

print("Metrics for CARLA vs UAH (aggressive):")
metrics_carla_uah_df = pd.DataFrame(metrics_carla_uah['aggressive']).T
display(metrics_carla_uah_df)

print("Metrics for SUMO vs UAH (normal):")
metrics_sumo_uah_df = pd.DataFrame(metrics_sumo_uah['normal']).T
display(metrics_sumo_uah_df)

print("Metrics for SUMO vs UAH (aggressive):")
metrics_sumo_uah_df = pd.DataFrame(metrics_sumo_uah['aggressive']).T
display(metrics_sumo_uah_df)

print("Metrics for UAH (driver 1) vs UAH (driver 2) (normal):")
metrics_uah_uah_df = pd.DataFrame(metrics_uah_uah['normal']).T
display(metrics_uah_uah_df)

print("Metrics for UAH (driver 1) vs UAH (driver 2) (aggressive):")
metrics_uah_uah_df = pd.DataFrame(metrics_uah_uah['aggressive']).T
display(metrics_uah_uah_df)


Metrics for CARLA vs SUMO (normal):


,KL Divergence ↓,Jensen-Shannon Distance ↓,Wasserstein Distance ↓,KS Statistic ↓,KS p-value ↑
acc_x,18.774264,0.776341,1.396831,0.529756,0.000000e+00
acc_y,3.082248,0.403015,0.153529,0.221663,1.328931e-200
gyro_z,6.682938,0.751490,0.033686,0.854464,0.000000e+00


Metrics for CARLA vs SUMO (aggressive):


,KL Divergence ↓,Jensen-Shannon Distance ↓,Wasserstein Distance ↓,KS Statistic ↓,KS p-value ↑
acc_x,19.897877,0.811230,1.401596,0.499681,5.859619e-321
acc_y,2.782651,0.380501,0.166034,0.224727,4.996152e-109
gyro_z,4.575010,0.645555,0.027641,0.798232,0.000000e+00


Metrics for CARLA vs UAH (normal):


...

,KL Divergence ↓,Jensen-Shannon Distance ↓,Wasserstein Distance ↓,KS Statistic ↓,KS p-value ↑
acc_x,0.329539,0.205755,0.016583,0.229097,2.607067e-97
acc_y,0.642446,0.178718,0.067987,0.112240,1.529580e-23
gyro_z,5.192663,0.813097,1.287961,0.661020,0.000000e+00


Metrics for UAH (driver 1) vs UAH (driver 2) (normal):


,KL Divergence ↓,Jensen-Shannon Distance ↓,Wasserstein Distance ↓,KS Statistic ↓,KS p-value ↑
acc_x,0.042077,0.069574,0.001848,0.021657,1.004593e-01
acc_y,0.274627,0.129350,0.006981,0.071306,1.879186e-14
acc_z,0.102040,0.080388,0.001666,0.040093,7.280650e-05
gyro_x,0.389288,0.167548,0.006887,0.071320,1.855523e-14
gyro_y,0.262337,0.177543,0.008459,0.113583,4.905236e-36
gyro_z,1.234432,0.450433,0.292296,0.254806,4.101792e-181


Metrics for UAH (driver 1) vs UAH (driver 2) (aggressive):


,KL Divergence ↓,Jensen-Shannon Distance ↓,Wasserstein Distance ↓,KS Statistic ↓,KS p-value ↑
acc_x,0.161144,0.112151,0.007119,0.054640,7.497024e-08
acc_y,0.389908,0.112884,0.005049,0.034577,2.104477e-03
acc_z,0.338515,0.282960,0.016846,0.182951,6.036083e-84
gyro_x,0.391495,0.308030,0.026823,0.317208,2.254116e-254
gyro_y,2.867717,0.301095,0.017855,0.121207,5.535696e-37
gyro_z,7.963827,0.540213,0.427332,0.321167,7.246466e-261


In [241]:
from scipy.spatial.distance import jensenshannon
from scipy.stats import gaussian_kde, energy_distance
from sklearn.metrics.pairwise import rbf_kernel
from scipy.spatial.distance import cdist, pdist
# Optional: fix random seed for reproducibility
np.random.seed(42)

In [242]:
def multivariate_energy_distance(X, Y):
    # pairwise distances between X and Y
    XY_dist = cdist(X, Y, metric='euclidean')
    # pairwise distances within X
    XX_dist = pdist(X, metric='euclidean')
    # pairwise distances within Y
    YY_dist = pdist(Y, metric='euclidean')

    return 2 * XY_dist.mean() - XX_dist.mean() - YY_dist.mean()

In [243]:
def compare_multivariate_distributions(data1, data2, bandwidth=0.2, n_eval_points=1000, seed=42):
    rng = np.random.default_rng(seed)

    # 1. Energy Distance
    energy_dist = multivariate_energy_distance(data1, data2)

    # 2. Jensen-Shannon Distance (approximate with KDE)
    def js_distance_kde(p, q):
        kde_p = gaussian_kde(p.T, bw_method=bandwidth)
        kde_q = gaussian_kde(q.T, bw_method=bandwidth)

        points = np.vstack([p, q])
        eval_pts = points[rng.choice(len(points), n_eval_points, replace=False)]

        p_eval = kde_p(eval_pts.T)
        q_eval = kde_q(eval_pts.T)

        # Normalize densities to sum to 1
        p_eval /= p_eval.sum()
        q_eval /= q_eval.sum()

        return jensenshannon(p_eval, q_eval)

    js_dist = js_distance_kde(data1, data2)

    # 3. Maximum Mean Discrepancy (MMD)
    def compute_mmd(X, Y, gamma=1.0):
        XX = rbf_kernel(X, X, gamma=gamma)
        YY = rbf_kernel(Y, Y, gamma=gamma)
        XY = rbf_kernel(X, Y, gamma=gamma)
        return XX.mean() + YY.mean() - 2 * XY.mean()

    mmd = compute_mmd(data1, data2, gamma=1.0)

    return {
        'energy_distance': energy_dist,
        'js_distance': js_dist,
        'mmd': mmd
    }

Now tring multivariate approaches to give SUMO and CARLA another chance

In [283]:
# Load or prepare your data
mult_carla_uah = {}
for id in carla.keys():
    beh = id.split('_')[1]
    if beh == 'normal':
        uah = df_acc['normal'].copy()
    else:
        uah = df_acc['aggressive'].copy()
    
    uah.rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'}, inplace=True)

    # Assuming carla_df and uah_df are your dataframes
    carla_acc =(carla[id][['acc_x', 'acc_y', 'acc_z']]).to_numpy(dtype=np.float64)
    uah_acc =(uah[['acc_x', 'acc_y', 'acc_z']]).to_numpy(dtype=np.float64)
    carla_gyro =(carla[id][['gyro_x', 'gyro_y', 'gyro_z']]).to_numpy(dtype=np.float64)
    uah_gyro =(uah[['gyro_x', 'gyro_y', 'gyro_z']]).to_numpy(dtype=np.float64)

    acc_results = compare_multivariate_distributions(carla_acc, uah_acc)
    gyro_results = compare_multivariate_distributions(carla_gyro, uah_gyro)

    mult_carla_uah[beh] = {
        'acc': acc_results,
        'gyro': gyro_results
    }

In [294]:
mult_sumo_uah = {}
for id in sumo.keys():
    beh = id.split('_')[1]
    if beh == 'normal':
        uah = df_acc['normal'].copy()
    else:
        uah = df_acc['aggressive'].copy()

    # Assuming carla_df and uah_df are your dataframes
    sumo_acc = np.array(sumo[id][['acc_x', 'acc_y']])
    uah_acc = np.array(uah[['acc_x', 'acc_y']])

    acc_results = compare_multivariate_distributions(sumo_acc, uah_acc)

    mult_sumo_uah[beh] = {
        'acc': acc_results,
    }

In [295]:
mult_carla_sumo = {}
for id in sumo.keys():
    beh = id.split('_')[1]
    # Assuming carla_df and uah_df are your dataframes
    sumo_acc = np.array(sumo[id][['acc_x', 'acc_y']])
    carla_acc = np.array(carla[id][['acc_x', 'acc_y']])

    acc_results = compare_multivariate_distributions(sumo_acc, carla_acc)

    mult_carla_sumo[beh] = {
        'acc': acc_results,
    }


In [289]:
mult_uah_uah = {}
for id in ['normal', 'aggressive']:
    if id == 'normal':
        uah_1 = df_acc['normal'].copy()
        uah_2 = df_acc_2['normal'].copy()
    else:
        uah_1 = df_acc['aggressive'].copy()
        uah_2 = df_acc_2['aggressive'].copy()

    uah_1.rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'}, inplace=True)
    uah_2.rename(columns={'Roll': 'gyro_x', 'Pitch': 'gyro_y', 'Yaw': 'gyro_z'}, inplace=True)

    # Assuming carla_df and uah_df are your dataframes
    uah_1_acc = np.array(uah_1[['acc_x', 'acc_y', 'acc_z']])
    uah_2_acc = np.array(uah_2[['acc_x', 'acc_y', 'acc_z']])

    uah_1_gyro = np.array(uah_1[['gyro_x', 'gyro_y', 'gyro_z']])
    uah_2_gyro = np.array(uah_2[['gyro_x', 'gyro_y', 'gyro_z']])

    acc_results = compare_multivariate_distributions(uah_1_acc, uah_2_acc)
    gyro_results = compare_multivariate_distributions(uah_1_gyro, uah_2_gyro)

    mult_uah_uah[id] = {
        'acc': acc_results,
        'gyro': gyro_results
    }

In [296]:
# Convert metrics dict to DataFrame for tabular display
print("Metrics for CARLA vs SUMO (normal):")
mult_carla_sumo_df = pd.DataFrame(mult_carla_sumo['normal']).T
display(mult_carla_sumo_df)

print("Metrics for CARLA vs SUMO (aggressive):")
mult_carla_sumo_df = pd.DataFrame(mult_carla_sumo['aggressive']).T
display(mult_carla_sumo_df)

print("Metrics for CARLA vs UAH (normal):")
mult_carla_uah_df = pd.DataFrame(mult_carla_uah['normal']).T
display(mult_carla_uah_df)

print("Metrics for CARLA vs UAH (aggressive):")
mult_carla_uah_df = pd.DataFrame(mult_carla_uah['aggressive']).T
display(mult_carla_uah_df)

print("Metrics for SUMO vs UAH (normal):")
mult_sumo_uah_df = pd.DataFrame(mult_sumo_uah['normal']).T
display(mult_sumo_uah_df)

print("Metrics for SUMO vs UAH (aggressive):")
mult_sumo_uah_df = pd.DataFrame(mult_sumo_uah['aggressive']).T
display(mult_sumo_uah_df)

print("Metrics for UAH (driver 1) vs UAH (driver 2) (normal):")
mult_uah_uah_df = pd.DataFrame(mult_uah_uah['normal']).T
display(mult_uah_uah_df)

print("Metrics for UAH (driver 1) vs UAH (driver 2) (aggressive):")
mult_uah_uah_df = pd.DataFrame(mult_uah_uah['aggressive']).T
display(mult_uah_uah_df)

Metrics for CARLA vs SUMO (normal):


,energy_distance,js_distance,mmd
acc,0.923721,0.478492,0.719215


Metrics for CARLA vs SUMO (aggressive):


,energy_distance,js_distance,mmd
acc,0.906415,0.647158,0.673408


Metrics for CARLA vs UAH (normal):


...

,energy_distance,js_distance,mmd
acc,0.007807,0.224182,0.001343


Metrics for UAH (driver 1) vs UAH (driver 2) (normal):


,energy_distance,js_distance,mmd
acc,0.000349,0.106709,0.000002
gyro,0.079359,0.429031,0.031248


Metrics for UAH (driver 1) vs UAH (driver 2) (aggressive):


,energy_distance,js_distance,mmd
acc,0.001289,0.214113,0.000012
gyro,0.183174,0.481947,0.132497
